## Hellow World using flask

In [3]:
import os
hello_world_script_file = os.path.join(os.path.pardir, 'src', 'models', 'hello_world_api.py')

In [6]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return "hello {0}".format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ..\src\models\hello_world_api.py


In [7]:
import json
import requests

In [9]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name':'Kutub'})
r = requests.post(url,data)

In [10]:
print(r.text)

hello Kutub


## Machine Learning API using flask

In [11]:
machine_learning_api_script_file = os.path.join(os.path.pardir, 'src', 'models', 'machine_learning_api.py')

In [18]:
%%writefile $machine_learning_api_script_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

model_path = os.path.join('models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath, 'rb'))
model = pickle.load(open(model_filepath, 'rb'))

columns = [ u'Age', u'Fare', u'FamilySize', \
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 

@app.route('/api', methods=['POST'])
def make_prediction():
    data = json.dumps(request.get_json(force=True))
    df = pd.read_json(data)
    passenger_ids = df['PassengerId'].ravel()
    actuals = df['Survived'].ravel()
    X = df[columns].as_matrix().astype('float')
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predicted': predictions, 'Actual': actuals})
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ..\src\models\machine_learning_api.py


### Invoking API using requests

In [19]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [20]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [21]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_Z,Pclass_1,Pclass_2,Pclass_3,Title_Lady,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1


In [22]:
import requests
def make_api_request(data):
    url = 'http://127.0.0.1:10001/api'
    r = requests.post(url,data)
    return r.json()

In [23]:
make_api_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [24]:
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

,PassengerId,Predicted,Actual
0,1,0,0
1,2,1,1
2,3,1,1
3,4,1,1
4,5,0,0


In [25]:
import numpy as np
np.mean(df_result.Actual == df_result.Predicted)

0.7755331088664422